In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_pandas
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

from copy import copy
import os
import sys
sys.path.insert(0, '../code/venv')
from qa_system import QuestionAnswerSystem
from utils import Utility

In [4]:
df = pd.read_pickle('tmp_save_me.pkl')
train_df_idxs, test_df_idxs = Utility.train_test_split(df)

In [5]:
def tmp(data):
    predict = np.array(list(data["sentence_lem"]))[np.argsort(list(data['train_predict_proba']))[::-1]][0]
    answer = data[data.answer_in_sentence == 1].sentence_lem.values
    return int(predict in answer)

In [7]:
target = 'answer_in_sentence'
predictors = df.columns.difference([
    'sentence',
    'sentence_lem',
    'question',
    'question_lem',
    'answer',
    'answer_lem',
    
    'question_ners',
    'sentence_ners',
    'question_type',
    target
])
df_train = df.loc[train_df_idxs].copy()
df_test = df.loc[test_df_idxs].copy()

sc = StandardScaler()
X_train_sc = sc.fit_transform(df_train[predictors])
X_test_sc = sc.transform(df_test[predictors])

clf = LogisticRegression()
clf.fit(X_train_sc, df_train[target])

df_train['train_predict_proba'] = clf.predict_proba(X_train_sc)[:, 1]
df_test['test_predict_proba'] = clf.predict_proba(X_test_sc)[:, 1]

tqdm_pandas(tqdm(total=df_train.index.nunique()))
train_score = df_train.groupby('question_id').progress_apply(lambda x: Utility.get_answer_by_score(x, 'train_predict_proba')).sum()/df_train.index.nunique()

tqdm_pandas(tqdm(total=df_test.index.nunique()))
test_score = df_test.groupby('question_id').progress_apply(lambda x: Utility.get_answer_by_score(x, 'test_predict_proba')).sum()/df_test.index.nunique()

scores = {'Train': train_score, 'Test': test_score}
scores

  0%|          | 0/700 [00:00<?, ?it/s]
701it [00:00, 1135.63it/s]                         
  0%|          | 0/300 [00:00<?, ?it/s]
301it [00:00, 1119.50it/s]                        


{'Test': 0.96799316387032375, 'Train': 0.97098895927659212}

In [8]:
df_train['answer_eq_predict'] = df_train.groupby('question_id').apply(lambda x: tmp(x))

In [9]:
df_train['max_train_predict_proba'] = df_train.groupby('question_id').apply(lambda x: x.train_predict_proba.max())

In [12]:
t1 = df_train[(df_train.answer_eq_predict == 0) &(df_train.max_train_predict_proba == df_train.train_predict_proba)][['question', 'sentence', 'question_ners', 'sentence_ners']]

In [13]:
t2 = df_train[(df_train.answer_in_sentence == 1) & (df_train.answer_eq_predict == 0)][['question', 'sentence', 'question_ners', 'sentence_ners']]

In [16]:
t1.columns = ['question', 'predict', 'question_ners', 'sentence_ners']

In [18]:
t2.columns = ['question', 'answer', 'question_ners', 'sentence_ners']

In [24]:
t3 = pd.merge(t1[['question', 'predict']].reset_index(), t2['answer'].reset_index(), how='left', on='question_id')

In [26]:
with open('sevochka_lulu.txt', 'w') as fout:
    for i, v in t3.iterrows():
        fout.write('question: {}\n'.format(v.question))
        fout.write('predict: {}\n'.format(v.predict))
        fout.write('answer: {}\n\n'.format(v.answer))